## Importing Dependencies

In [ ]:
import os
import json
import shutil
import random
from pathlib import Path
from collections import defaultdict

## Sampling Videos from particular video tracks
This is done because these video images have feasible object detection and are at considerable distances from the drone other images consists of object that are very very far away from the drone and appear very very small.

In [ ]:
# ===== CONFIG =====
IMG_DIR = r"D:\Dataset\images\train"
JSON_PATH = "./dataset/seadronessee/annotations/instances_train_objects_in_water_life_jacket_rm_fixed.json"
DEST_IMG_DIR = "./dataset/seadronessee_sampled/images/train"
DEST_JSON_PATH = "./dataset/seadronessee_sampled/annotations/train.json"
TARGET_SIZE_GB = 10  # Stop when ~10GB is reached

# Only include these video IDs
ALLOWED_VIDEO_IDS = {0, 5, 7, 8, 9, 12, 13, 14, 15, 18, 19, 21}
# ==================

os.makedirs(DEST_IMG_DIR, exist_ok=True)
os.makedirs(os.path.dirname(DEST_JSON_PATH), exist_ok=True)

# Load JSON
with open(JSON_PATH, 'r') as f:
    data = json.load(f)

# Group images by video_id (only allowed ones)
video_images = defaultdict(list)
image_id_map = {}

for img in data['images']:
    if img['video_id'] in ALLOWED_VIDEO_IDS:
        video_images[img['video_id']].append(img)
        image_id_map[img['id']] = img

# Randomize order in each video
for vid in video_images:
    random.shuffle(video_images[vid])

# Start sampling
total_bytes = 0
selected_images = []
selected_image_ids = set()

video_ids = sorted(video_images.keys())
while total_bytes < TARGET_SIZE_GB * (1024**3) and any(video_images.values()):
    for vid in video_ids:
        if video_images[vid] and total_bytes < TARGET_SIZE_GB * (1024**3):
            img = video_images[vid].pop()  # take one image
            src_path = Path(IMG_DIR) / img['file_name']
            file_size = src_path.stat().st_size

            if total_bytes + file_size > TARGET_SIZE_GB * (1024**3):
                break

            # Copy image
            shutil.copy2(src_path, Path(DEST_IMG_DIR) / img['file_name'])
            selected_images.append(img)
            selected_image_ids.add(img['id'])
            total_bytes += file_size

# Filter annotations for selected images
selected_annotations = [ann for ann in data['annotations'] if ann['image_id'] in selected_image_ids]

# Keep only selected videos
selected_video_ids = {img['video_id'] for img in selected_images}
selected_videos = [vid for vid in data['videos'] if vid['id'] in selected_video_ids]

# Save filtered JSON
filtered_data = data.copy()
filtered_data['images'] = selected_images
filtered_data['annotations'] = selected_annotations
filtered_data['videos'] = selected_videos

with open(DEST_JSON_PATH, 'w') as f:
    json.dump(filtered_data, f, indent=4)

print(f"✅ Done! {len(selected_images)} images selected from {len(selected_video_ids)} videos, total size: {total_bytes / (1024**3):.2f} GB")


In [ ]:
# ===== CONFIG =====
IMG_DIR = r"D:\Dataset\images\val"
JSON_PATH = "./dataset/seadronessee/annotations/instances_val_objects_in_fixed.json"
DEST_IMG_DIR = "./dataset/seadronessee_sampled/images/val"
DEST_JSON_PATH = "./dataset/seadronessee_sampled/annotations/val.json"
TARGET_SIZE_GB = 2  # Stop when ~10GB is reached

# Only include these video IDs
ALLOWED_VIDEO_IDS = {0, 5, 9, 11, 12, 13, 15, 17, 18, 19, 21}
# ==================

os.makedirs(DEST_IMG_DIR, exist_ok=True)
os.makedirs(os.path.dirname(DEST_JSON_PATH), exist_ok=True)

# Load JSON
with open(JSON_PATH, 'r') as f:
    data = json.load(f)

# Group images by video_id (only allowed ones)
video_images = defaultdict(list)
image_id_map = {}

for img in data['images']:
    if img['video_id'] in ALLOWED_VIDEO_IDS:
        video_images[img['video_id']].append(img)
        image_id_map[img['id']] = img

# Randomize order in each video
for vid in video_images:
    random.shuffle(video_images[vid])

# Start sampling
total_bytes = 0
selected_images = []
selected_image_ids = set()

video_ids = sorted(video_images.keys())
while total_bytes < TARGET_SIZE_GB * (1024**3) and any(video_images.values()):
    for vid in video_ids:
        if video_images[vid] and total_bytes < TARGET_SIZE_GB * (1024**3):
            img = video_images[vid].pop()  # take one image
            src_path = Path(IMG_DIR) / img['file_name']
            file_size = src_path.stat().st_size

            if total_bytes + file_size > TARGET_SIZE_GB * (1024**3):
                break

            # Copy image
            shutil.copy2(src_path, Path(DEST_IMG_DIR) / img['file_name'])
            selected_images.append(img)
            selected_image_ids.add(img['id'])
            total_bytes += file_size

# Filter annotations for selected images
selected_annotations = [ann for ann in data['annotations'] if ann['image_id'] in selected_image_ids]

# Keep only selected videos
selected_video_ids = {img['video_id'] for img in selected_images}
selected_videos = [vid for vid in data['videos'] if vid['id'] in selected_video_ids]

# Save filtered JSON
filtered_data = data.copy()
filtered_data['images'] = selected_images
filtered_data['annotations'] = selected_annotations
filtered_data['videos'] = selected_videos

with open(DEST_JSON_PATH, 'w') as f:
    json.dump(filtered_data, f, indent=4)

print(f"✅ Done! {len(selected_images)} images selected from {len(selected_video_ids)} videos, total size: {total_bytes / (1024**3):.2f} GB")

## Preprocessing the annotation and creating YOLO format annotation
The "swimmer" and "swimmer with life jacket" are merged as "person" class to make it easy because in night time using IR footages it will be difficult to distinguish between a "swimmer" and a "swimmer with life jacket" so that's why.

In [ ]:
def merge_and_convert_coco_to_yolo(coco_json_path, images_dir, output_labels_dir, output_coco_json_path):
    """
    Merge 'swimmer' and 'swimmers with life jacket' into 'person'
    Keep only 'person' and 'boat' categories
    Save new COCO JSON and YOLO txt label files
    """
    
    os.makedirs(output_labels_dir, exist_ok=True)
    
    print(f"Loading COCO annotations from {coco_json_path}...")
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)
    
    # ==============================================================
    # 🟢 Step 1: Identify categories
    # ==============================================================
    merge_classes = ["swimmer", "swimmers with life jacket"]
    merged_class_name = "person"
    keep_class = "boat"
    
    # Find IDs for the classes we’ll merge
    merge_ids = [cat['id'] for cat in coco['categories'] if cat['name'].lower() in merge_classes]
    boat_ids = [cat['id'] for cat in coco['categories'] if cat['name'].lower() == keep_class]
    
    if not merge_ids or not boat_ids:
        raise ValueError("Could not find 'swimmer'/'swimmers with life jacket' or 'boat' categories in JSON.")
    
    print(f"Merging {merge_classes} into '{merged_class_name}'...")
    
    # New category mapping (COCO old id → new id)
    # 1 -> person
    # 2 -> boat
    category_mapping = {}
    for cid in merge_ids:
        category_mapping[cid] = 1  # person
    for cid in boat_ids:
        category_mapping[cid] = 2  # boat
    
    # ==============================================================
    # 🟢 Step 2: Filter and update annotations
    # ==============================================================
    new_annotations = []
    ann_id = 1
    for ann in coco['annotations']:
        old_cid = ann['category_id']
        if old_cid not in category_mapping:
            continue  # skip unwanted classes
        
        new_ann = ann.copy()
        new_ann['category_id'] = category_mapping[old_cid]
        new_ann['id'] = ann_id
        new_annotations.append(new_ann)
        ann_id += 1
    
    # ==============================================================
    # 🟢 Step 3: Create new categories list
    # ==============================================================
    new_categories = [
        {"id": 1, "name": "person"},
        {"id": 2, "name": "boat"}
    ]
    
    # ==============================================================
    # 🟢 Step 4: Build new COCO dict and save
    # ==============================================================
    new_coco = {
        "images": coco['images'],
        "annotations": new_annotations,
        "categories": new_categories
    }
    
    os.makedirs(os.path.dirname(output_coco_json_path), exist_ok=True)
    with open(output_coco_json_path, 'w') as f:
        json.dump(new_coco, f)
    
    print(f"✅ New COCO file saved: {output_coco_json_path}")
    print(f"   Contains {len(new_annotations)} annotations and 2 categories.")
    
    # ==============================================================
    # 🟢 Step 5: Convert to YOLO txt labels
    # ==============================================================
    annotations_by_image = {}
    for ann in new_annotations:
        annotations_by_image.setdefault(ann['image_id'], []).append(ann)
    
    converted_count = 0
    skipped_count = 0
    
    for image in coco['images']:
        file_name = image['file_name']
        width, height = image['width'], image['height']
        txt_path = os.path.join(output_labels_dir, Path(file_name).stem + '.txt')
        
        anns = annotations_by_image.get(image['id'], [])
        if not anns:
            open(txt_path, 'w').close()
            skipped_count += 1
            continue
        
        yolo_lines = []
        for ann in anns:
            cat_id = ann['category_id']
            class_id = cat_id - 1  # YOLO is 0-indexed: person→0, boat→1
            
            x, y, w, h = ann['bbox']
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height
            
            yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
        
        with open(txt_path, 'w') as f:
            f.write("\n".join(yolo_lines))
        
        converted_count += 1
    
    # ==============================================================
    # 🟢 Step 6: Save classes.txt
    # ==============================================================
    with open(os.path.join(output_labels_dir, "classes.txt"), 'w') as f:
        f.write("person\nboat\n")
    
    print(f"\n✅ Conversion complete!")
    print(f"Converted: {converted_count} images with annotations")
    print(f"Skipped: {skipped_count} images without annotations")
    print(f"Output directory: {output_labels_dir}")
    print(f"YOLO classes saved to: {os.path.join(output_labels_dir, 'classes.txt')}")

In [ ]:
# ======================================================================
# Usage Example
# ======================================================================

train_json = "./dataset/seadronessee_sampled/annotations/train.json"
val_json = "./dataset/seadronessee_sampled/annotations/val.json"

train_images = "./dataset/seadronessee_sampled/images/train/"
val_images = "./dataset/seadronessee_sampled/images/val/"

train_labels_output = "./dataset/seadronessee_sampled/labels/train"
val_labels_output = "./dataset/seadronessee_sampled/labels/val"

train_new_json = "./dataset/seadronessee_sampled/annotations/train_2cat.json"
val_new_json = "./dataset/seadronessee_sampled/annotations/val_2cat.json"

print("=" * 60)
print("Converting TRAIN annotations...")
print("=" * 60)
merge_and_convert_coco_to_yolo(train_json, train_images, train_labels_output, train_new_json)

print("\n" + "=" * 60)
print("Converting VAL annotations...")
print("=" * 60)
merge_and_convert_coco_to_yolo(val_json, val_images, val_labels_output, val_new_json)

In [ ]:
# ======================================================================
# Usage Example
# ======================================================================

train_json = "./dataset/seadronessee_sampled/annotations/train.json"
val_json = "./dataset/seadronessee_sampled/annotations/val.json"

train_images = "./dataset/seadronessee_sampled/images/train/"
val_images = "./dataset/seadronessee_sampled/images/val/"

train_labels_output = "./dataset/seadronessee_sampled/labels/train"
val_labels_output = "./dataset/seadronessee_sampled/labels/val"

train_new_json = "./dataset/seadronessee_sampled/annotations/train_2cat.json"
val_new_json = "./dataset/seadronessee_sampled/annotations/val_2cat.json"

print("=" * 60)
print("Converting TRAIN annotations...")
print("=" * 60)
merge_and_convert_coco_to_yolo(train_json, train_images, train_labels_output, train_new_json)

print("\n" + "=" * 60)
print("Converting VAL annotations...")
print("=" * 60)
merge_and_convert_coco_to_yolo(val_json, val_images, val_labels_output, val_new_json)

## Converting images from RGBA to RGB

In [1]:
# --- Input and Output folders ---
input_dir = "./dataset/SeaDronesSee_IR/images/train/"
output_dir = "./dataset/SeaDronesSee_IR_processed/images/train"

# --- Create output directory if it doesn't exist ---
os.makedirs(output_dir, exist_ok=True)

# --- Loop through all PNG files ---
for file in os.listdir(input_dir):
    if file.endswith(".png"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)

        # Open, convert, and save to new folder
        img = Image.open(input_path).convert("RGB")
        img.save(output_path)

print("✅ All images converted to RGB and saved to:", output_dir)


✅ All images converted to RGB and saved to: ./dataset/SeaDronesSee_IR_processed/images/train


In [2]:
# --- Input and Output folders ---
input_dir = "./dataset/SeaDronesSee_IR/images/val/"
output_dir = "./dataset/SeaDronesSee_IR_processed/images/val"

# --- Create output directory if it doesn't exist ---
os.makedirs(output_dir, exist_ok=True)

# --- Loop through all PNG files ---
for file in os.listdir(input_dir):
    if file.endswith(".png"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)

        # Open, convert, and save to new folder
        img = Image.open(input_path).convert("RGB")
        img.save(output_path)

print("✅ All images converted to RGB and saved to:", output_dir)


✅ All images converted to RGB and saved to: ./dataset/SeaDronesSee_IR_processed/images/val
